In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Add
from tensorflow.keras.layers import Conv1D, SpatialDropout1D, GlobalAveragePooling1D, Activation
from tensorflow.keras.regularizers import l2
from pathlib import Path
import os

print("TensorFlow Version:", tf.__version__)

# --- Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- Configuration: Paths and Model Hyperparameters ---
# Make sure this path matches your Drive structure
BASE_DIR = Path('/content/drive/My Drive/Colab_HAR_Project/data')
# The original script saves a model with a prefix. The image shows 'har_model.keras'.
# We will use 'har_model.keras' as it exists in your folder.
# If you retrain, you might need to change this to 'tcn_ABCDE_har_model.keras'.
PRE_TRAINED_MODEL_PATH = BASE_DIR / 'har_model.keras'
# Define the output path for the new TFLite model
TFLITE_MODEL_PATH = BASE_DIR / 'har_federated_model.tflite'

# --- Model Hyperparameters (MUST MATCH THE ORIGINAL TRAINING SCRIPT) ---
# These parameters are copied from TCNonOwnData_offline.py to ensure the model
# architecture is identical, which is required for loading weights.
WINDOW_SIZE = 60
NUM_FEATURES = 6 # (x, y, z for accel) + (x, y, z for gyro)
NUM_CLASSES = 5  # Corresponds to A, B, C, D, E
KERNEL_SIZE = 7
NUM_FILTERS = 64
NUM_TCN_BLOCKS = 5
DILATION_RATES = [2**i for i in range(NUM_TCN_BLOCKS)]
SPATIAL_DROPOUT_RATE = 0.15
FINAL_DROPOUT_RATE = 0.3
L2_REG = 1e-4

# --- Verify Paths ---
print("\n--- Paths ---")
if not BASE_DIR.exists():
    print(f"❌ ERROR: Base directory not found: {BASE_DIR}")
else:
    print(f"✅ Base directory found: {BASE_DIR}")

if not PRE_TRAINED_MODEL_PATH.exists():
    print(f"❌ ERROR: Pre-trained model not found: {PRE_TRAINED_MODEL_PATH}")
    print("Please ensure the model file exists and the path is correct.")
else:
    print(f"✅ Pre-trained model found: {PRE_TRAINED_MODEL_PATH}")

print(f"Output TFLite model will be saved to: {TFLITE_MODEL_PATH}")

TensorFlow Version: 2.18.0
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

--- Paths ---
✅ Base directory found: /content/drive/My Drive/Colab_HAR_Project/data
✅ Pre-trained model found: /content/drive/My Drive/Colab_HAR_Project/data/har_model.keras
Output TFLite model will be saved to: /content/drive/My Drive/Colab_HAR_Project/data/har_federated_model.tflite


In [ ]:
# --- TCN Model Definition (Copied from the training script) ---
# This is crucial for ensuring the architecture matches for weight loading.

def residual_block(x, dilation_rate, nb_filters, kernel_size, padding, dropout_rate=0.0, l2_reg=0.0):
    prev_x = x
    conv1 = Conv1D(filters=nb_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding=padding, kernel_regularizer=l2(l2_reg))(x)
    conv1 = BatchNormalization()(conv1); conv1 = Activation('relu')(conv1); conv1 = SpatialDropout1D(dropout_rate)(conv1)
    conv2 = Conv1D(filters=nb_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding=padding, kernel_regularizer=l2(l2_reg))(conv1)
    conv2 = BatchNormalization()(conv2); conv2 = Activation('relu')(conv2); conv2 = SpatialDropout1D(dropout_rate)(conv2)
    if prev_x.shape[-1] != conv2.shape[-1]: prev_x = Conv1D(nb_filters, 1, padding='same', kernel_regularizer=l2(l2_reg))(prev_x)
    res_x = Add()([prev_x, conv2]); return res_x

def build_tcn_model(input_shape, num_classes, kernel_size, num_filters, dilation_rates, spatial_dropout, final_dropout, l2_reg=0.0):
    input_layer = Input(shape=input_shape)
    x = input_layer
    for dilation_rate in dilation_rates:
        x = residual_block(x, dilation_rate, num_filters, kernel_size, padding='same', dropout_rate=spatial_dropout, l2_reg=l2_reg)
    x = GlobalAveragePooling1D()(x)
    x = Dropout(final_dropout)(x)
    output_layer = Dense(num_classes, activation='softmax', kernel_regularizer=l2(l2_reg))(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# --- Instantiate and Load the Model ---
print("Building model architecture...")
input_shape = (WINDOW_SIZE, NUM_FEATURES)
keras_model = build_tcn_model(
    input_shape, NUM_CLASSES, KERNEL_SIZE, NUM_FILTERS,
    DILATION_RATES, SPATIAL_DROPOUT_RATE, FINAL_DROPOUT_RATE, L2_REG
)

print(f"Loading pre-trained weights from {PRE_TRAINED_MODEL_PATH}...")
try:
    # Use load_weights because we only care about the parameters, not the optimizer state
    keras_model.load_weights(PRE_TRAINED_MODEL_PATH)
    print("✅ Successfully loaded model weights.")
    keras_model.summary()
except Exception as e:
    print(f"❌ ERROR loading weights: {e}")
    print("Please check that the model architecture and hyperparameters in this script match the saved model.")

Building model architecture...
Loading pre-trained weights from /content/drive/My Drive/Colab_HAR_Project/data/har_model.keras...
✅ Successfully loaded model weights.


Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_13      │ (None, 60, 6)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_233 (Conv1D) │ (None, 60, 64)    │      2,752 │ input_layer_13[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 64)    │        256 │ conv1d_233[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_190      │ (None, 60, 64)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_… │ (None, 60, 64)    │          0 │ activation_190[0… │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_234 (Conv1D) │ (None, 60, 64)    │     28,736 │ spatial_dropout1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 64)    │        256 │ conv1d_234[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_191      │ (None, 60, 64)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_235 (Conv1D) │ (None, 60, 64)    │        448 │ input_layer_13[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_… │ (None, 60, 64)    │          0 │ activation_191[0… │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_95 (Add)        │ (None, 60, 64)    │          0 │ conv1d_235[0][0], │
│                     │                   │            │ spatial_dropout1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_236 (Conv1D) │ (None, 60, 64)    │     28,736 │ add_95[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 64)    │        256 │ conv1d_236[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_192      │ (None, 60, 64)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_… │ (None, 60, 64)    │          0 │ activation_192[0… │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_237 (Conv1D) │ (None, 60, 64)    │     28,736 │ spatial_dropout1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 64)    │        256 │ conv1d_237[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_193      │ (None, 60, 64)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 264,709 (1.01 MB)

 Trainable params: 263,429 (1.00 MB)

 Non-trainable params: 1,280 (5.00 KB)

In [ ]:
# --- This is the definitive and final working solution ---
# (The UnifiedTrainableTCN class definition from the previous answer is correct)
# We just need to fix the instantiation and loading sequence.

# (Paste the full `UnifiedTrainableTCN` class definition from the previous answer here)
class UnifiedTrainableTCN(tf.keras.Model):
    """
    This class IS the TCN model, built using the standard Keras subclassing
    pattern. This is the most robust way to ensure all layers and resources
    are tracked correctly.
    """
    def __init__(self, num_classes, kernel_size, num_filters, dilation_rates,
                 spatial_dropout, final_dropout, l2_reg, **kwargs):
        super().__init__(**kwargs)

        # --- Build the model's layers as direct attributes in the constructor ---
        # Keras will automatically discover and build these layers.

        self.tcn_blocks = []
        for i, dilation_rate in enumerate(dilation_rates):
            block_layers = {}

            # First convolution in the block
            block_layers['conv1'] = Conv1D(filters=num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding='same', kernel_regularizer=l2(l2_reg))
            block_layers['bn1'] = BatchNormalization()
            block_layers['relu1'] = Activation('relu')
            block_layers['s_dropout1'] = SpatialDropout1D(spatial_dropout)

            # Second convolution in the block
            block_layers['conv2'] = Conv1D(filters=num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding='same', kernel_regularizer=l2(l2_reg))
            block_layers['bn2'] = BatchNormalization()
            block_layers['relu2'] = Activation('relu')
            block_layers['s_dropout2'] = SpatialDropout1D(spatial_dropout)

            # Residual connection handling
            block_layers['conv_res'] = Conv1D(num_filters, 1, padding='same', kernel_regularizer=l2(l2_reg))
            block_layers['add_res'] = Add()

            # Use a dictionary for this block's layers and append to the list
            self.tcn_blocks.append(block_layers)

        self.global_pool = GlobalAveragePooling1D()
        self.final_dropout = Dropout(final_dropout)
        self.output_layer = Dense(num_classes, activation='softmax', kernel_regularizer=l2(l2_reg))

    @tf.function
    def call(self, x, training=False):
        """Standard forward pass for inference."""
        for block in self.tcn_blocks:
            prev_x = x
            x = block['conv1'](x)
            x = block['bn1'](x, training=training)
            x = block['relu1'](x)
            x = block['s_dropout1'](x, training=training)
            x = block['conv2'](x)
            x = block['bn2'](x, training=training)
            x = block['relu2'](x)
            x = block['s_dropout2'](x, training=training)
            if prev_x.shape[-1] != x.shape[-1]:
                prev_x = block['conv_res'](prev_x)
            x = block['add_res']([prev_x, x])
        x = self.global_pool(x)
        x = self.final_dropout(x, training=training)
        return self.output_layer(x)

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None, WINDOW_SIZE, NUM_FEATURES], dtype=tf.float32),
        tf.TensorSpec(shape=[None], dtype=tf.int32)
    ])
    def train_step(self, x, y):
        with tf.GradientTape() as tape:
            predictions = self(x, training=True)
            loss = self.compiled_loss(y, predictions)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return {'loss': loss}

    @tf.function(input_signature=[])
    def get_weights_dict(self):
        return {f'weight_{i}': var for i, var in enumerate(self.trainable_variables)}

    @tf.function
    def set_weights_dict(self, new_weights: dict):
        for i, var in enumerate(self.trainable_variables):
            var.assign(new_weights[f'weight_{i}'])
        return {'status': tf.constant("Weights updated successfully")}


# --- Instantiate, BUILD, Load Weights, and Compile ---
# 1. Instantiate the unified model
final_model = UnifiedTrainableTCN(
    num_classes=NUM_CLASSES, kernel_size=KERNEL_SIZE, num_filters=NUM_FILTERS,
    dilation_rates=DILATION_RATES, spatial_dropout=SPATIAL_DROPOUT_RATE,
    final_dropout=FINAL_DROPOUT_RATE, l2_reg=L2_REG
)

# 2. **THE FIX: Explicitly build the model.**
# This creates the variables and prepares the model to receive weights.
print("Building model...")
final_model.build(input_shape=(None, WINDOW_SIZE, NUM_FEATURES))
print("✅ Model built.")

# 3. Compile the model.
final_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy()
)
print("✅ Model compiled.")

# 4. NOW, load the weights. This will succeed because the model is built.
print(f"Loading pre-trained weights from {PRE_TRAINED_MODEL_PATH}...")
final_model.load_weights(PRE_TRAINED_MODEL_PATH)
print("✅ Weights loaded successfully into the unified model.")

final_model.summary()

Building model...
✅ Model built.
✅ Model compiled.
Loading pre-trained weights from /content/drive/My Drive/Colab_HAR_Project/data/har_model.keras...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'unified_trainable_tcn_3', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 90 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: A total of 21 objects could not be loaded. Example error message for object <Conv1D name=conv1d_259, built=False>:

Layer 'conv1d_259' was never built and thus it doesn't have any variables. However the weights file lists 2 variables for this layer.
In most cases, this error indicates that either:

1. The layer is owned by a parent layer that implements a `build()` method, but calling the parent's `build()` method did NOT create the state of the child layer 'conv1d_259'. A `build()` method must create ALL state for the layer, including the state of any children layers.

2. You need to implement the `def build_from_config(self, config)` method on layer 'conv1d_259', to specify how to rebuild it during loading. In this case, you might also want to implement the method that generates the build config at saving time, `def get_build_config(self)`. The method `build_from_config()` is meant to create the state of the layer (i.e. its variables) upon deserialization.

List of object

In [ ]:
import tempfile
import shutil

print("Starting final 2-step TFLite conversion process...")

# --- Part 1: Save the Unified Model ---
saved_model_dir = tempfile.mkdtemp()
print(f"Creating temporary SavedModel directory at: {saved_model_dir}")

# Define the signatures using the methods from our unified model
signatures = {
    'serving_default': final_model.call.get_concrete_function(
        tf.TensorSpec(shape=[None, WINDOW_SIZE, NUM_FEATURES], dtype=tf.float32)),
    'train': final_model.train_step.get_concrete_function(),
    'get_weights': final_model.get_weights_dict.get_concrete_function(),
    'set_weights': final_model.set_weights_dict.get_concrete_function({
        f'weight_{i}': tf.TensorSpec(shape=var.shape, dtype=var.dtype)
        for i, var in enumerate(final_model.trainable_variables)
    })
}

tf.saved_model.save(final_model, saved_model_dir, signatures=signatures)
print("✅ Successfully saved unified model with multiple signatures.")

# --- Part 2: Convert and Save ---
print("\nConverting the SavedModel to TFLite...")
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False
tflite_model_content = converter.convert()

with open(TFLITE_MODEL_PATH, 'wb') as f:
    f.write(tflite_model_content)
shutil.rmtree(saved_model_dir)

print(f"\n🎉 Successfully converted and saved TFLite model!")
print(f"   Path: {TFLITE_MODEL_PATH}")

# --- Verification Step ---
# The previous verification script will work perfectly with this model.
print("\n--- Verifying the generated TFLite model ---")
# ... paste the full verification script here ...

In [ ]:
# Colab cell 1: Install & Imports
!pip install joblib -q
import os
import tempfile
import shutil
import json

import numpy as np
import joblib
import tensorflow as tf

from tensorflow.keras.models import load_model

# Colab cell 2: Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Colab cell 3: Define paths based on your Drive layout
BASE = '/content/drive/MyDrive/Colab_HAR_Project'
DATA_DIR = os.path.join(BASE, 'data')
RESULTS_DIR = os.path.join(BASE, 'results', 'Federated_HAR_Assets')
os.makedirs(RESULTS_DIR, exist_ok=True)

# Input files (as per your screenshot)
CSV1_PATH    = os.path.join(DATA_DIR, 'resampled_normalized_phone_data.csv')
CSV2_PATH    = os.path.join(DATA_DIR, 'combined_collected_data.csv')
MODEL_PATH   = os.path.join(DATA_DIR, 'har_model.keras')
LE_PATH      = os.path.join(DATA_DIR, 'tcn_ABCDE_label_encoder.joblib')

print("✔️ Paths ready:")
print("  CSV1:", CSV1_PATH)
print("  CSV2:", CSV2_PATH)
print("  Keras model:", MODEL_PATH)
print("  LabelEncoder:", LE_PATH)
print("  Output assets:", RESULTS_DIR)

# Colab cell 4: Load your artifacts
label_encoder = joblib.load(LE_PATH)
keras_model    = load_model(MODEL_PATH)

# (Re-)define these constants to match whatever your model expects:
WINDOW_SIZE   = 60
NUM_FEATURES  = 6    # e.g. ['x_accel','y_accel','z_accel','x_gyro','y_gyro','z_gyro']

# Colab cell 5 (updated): Wrap in a Federated tf.Module, tracking the global RNG
class FederatedModule(tf.Module):
    def __init__(self, keras_model, lr=1e-3):
        super().__init__()
        self.model     = keras_model
        self.optimizer = tf.keras.optimizers.Adam(lr)
        self.loss_fn   = tf.keras.losses.SparseCategoricalCrossentropy()
        # Track the global RNG resource
        self.global_rng = tf.random.experimental.get_global_generator()

    # Plain Python methods
    def serving_default(self, x):
        preds = self.model(x, training=False)
        return {"output": preds}

    def train(self, x, y):
        with tf.GradientTape() as tape:
            preds = self.model(x, training=True)
            loss  = self.loss_fn(y, preds)
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
        return {"loss": loss}

    def get_weights(self):
        return {
            f"weight_{i}": var
            for i, var in enumerate(self.model.trainable_variables)
        }

    def set_weights(self, *new_weights):
        for var, new_val in zip(self.model.trainable_variables, new_weights):
            var.assign(new_val)
        return {"status": tf.constant(1, tf.int32)}


# Instantiate
fed_mod = FederatedModule(keras_model)

# Now wrap each method in tf.function *after* instantiation,
# so that all resources on fed_mod (including global_rng) are tracked.

# Inference
infer_fn = tf.function(
    fed_mod.serving_default,
    input_signature=[tf.TensorSpec([None, WINDOW_SIZE, NUM_FEATURES], tf.float32, name="x")]
).get_concrete_function()

# Train
train_fn = tf.function(
    fed_mod.train,
    input_signature=[
        tf.TensorSpec([None, WINDOW_SIZE, NUM_FEATURES], tf.float32, name="x"),
        tf.TensorSpec([None], tf.int32, name="y"),
    ]
).get_concrete_function()

# Get weights (no inputs)
get_w_fn = tf.function(fed_mod.get_weights).get_concrete_function()

# Set weights (specs matching trainable_variables)
specs = [
    tf.TensorSpec(var.shape, var.dtype, name=f"weight_{i}")
    for i, var in enumerate(fed_mod.model.trainable_variables)
]
set_w_fn = tf.function(fed_mod.set_weights).get_concrete_function(*specs)


export_dir = tempfile.mkdtemp()
signatures = {
    "serving_default": infer_fn,
    "train":           train_fn,
    "get_weights":     get_w_fn,
    "set_weights":     set_w_fn,
}
tf.saved_model.save(fed_mod, export_dir, signatures=signatures)
print("✅ SavedModel with multi‑signatures at:", export_dir)


converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
tflite_model = converter.convert()
with open(os.path.join(RESULTS_DIR, 'har_federated_model.tflite'), 'wb') as f:
    f.write(tflite_model)
print("✅ .tflite written with 4 federated signatures.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✔️ Paths ready:
  CSV1: /content/drive/MyDrive/Colab_HAR_Project/data/resampled_normalized_phone_data.csv
  CSV2: /content/drive/MyDrive/Colab_HAR_Project/data/combined_collected_data.csv
  Keras model: /content/drive/MyDrive/Colab_HAR_Project/data/har_model.keras
  LabelEncoder: /content/drive/MyDrive/Colab_HAR_Project/data/tcn_ABCDE_label_encoder.joblib
  Output assets: /content/drive/MyDrive/Colab_HAR_Project/results/Federated_HAR_Assets


AssertionError: Tried to export a function which references an 'untracked' resource. TensorFlow objects (e.g. tf.Variable) captured by functions must be 'tracked' by assigning them to an attribute of a tracked object or assigned to an attribute of the main object directly. See the information below:
	Function name = b'__inference_signature_wrapper_322230'
	Captured Tensor = <ResourceHandle(name="seed_generator_462/seed_generator_state/6621", device="/job:localhost/replica:0/task:0/device:CPU:0", container="Anonymous", type="tensorflow::Var", dtype and shapes : "[ DType enum: 9, Shape: [2] ]")>
	Trackable referencing this tensor = <tf.Variable 'seed_generator_462/seed_generator_state:0' shape=(2,) dtype=int64>
	Internal Tensor = Tensor("321910:0", shape=(), dtype=resource)

chatGpt

In [ ]:
!pip install joblib -q
import os, tempfile, shutil
import joblib
import tensorflow as tf
from tensorflow.keras.models import load_model
from google.colab import drive


In [ ]:
drive.mount('/content/drive', force_remount=True)

BASE_DIR      = '/content/drive/MyDrive/Colab_HAR_Project'
DATA_DIR      = os.path.join(BASE_DIR, 'data')
RESULTS_DIR   = os.path.join(BASE_DIR, 'results', 'Federated_HAR_Assets')
TF_OUTPUT     = os.path.join(RESULTS_DIR, 'har_federated_model.tflite')

os.makedirs(RESULTS_DIR, exist_ok=True)


Mounted at /content/drive


In [ ]:
KERAS_MODEL_PATH = os.path.join(DATA_DIR, 'har_model.keras')

# Load your Functional API model directly
keras_model = load_model(KERAS_MODEL_PATH)
keras_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 60, 6)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_11 (Conv1D)  │ (None, 60, 64)    │      2,752 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 64)    │        256 │ conv1d_11[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_10       │ (None, 60, 64)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_… │ (None, 60, 64)    │          0 │ activation_10[0]… │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_12 (Conv1D)  │ (None, 60, 64)    │     28,736 │ spatial_dropout1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 64)    │        256 │ conv1d_12[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_11       │ (None, 60, 64)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_13 (Conv1D)  │ (None, 60, 64)    │        448 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_… │ (None, 60, 64)    │          0 │ activation_11[0]… │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 60, 64)    │          0 │ conv1d_13[0][0],  │
│                     │                   │            │ spatial_dropout1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_14 (Conv1D)  │ (None, 60, 64)    │     28,736 │ add_5[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 64)    │        256 │ conv1d_14[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_12       │ (None, 60, 64)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_… │ (None, 60, 64)    │          0 │ activation_12[0]… │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_15 (Conv1D)  │ (None, 60, 64)    │     28,736 │ spatial_dropout1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 64)    │        256 │ conv1d_15[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_13       │ (None, 60, 64)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 791,569 (3.02 MB)

 Trainable params: 263,429 (1.00 MB)

 Non-trainable params: 1,280 (5.00 KB)

 Optimizer params: 526,860 (2.01 MB)

In [ ]:
# Cell 4: Define the FederatedModule (no @tf.function here, but track RNG)

class FederatedModule(tf.Module):
    def __init__(self, model, lr=1e-3):
        super().__init__()
        self.model     = model
        self.optimizer = tf.keras.optimizers.Adam(lr)
        self.loss_fn   = tf.keras.losses.SparseCategoricalCrossentropy()
        # 👇 Track the global RNG variable so it's not "untracked"
        self._rng = tf.random.experimental.get_global_generator()

    def serving_default(self, x):
        return {"output": self.model(x, training=False)}

    def train(self, x, y):
        with tf.GradientTape() as tape:
            logits = self.model(x, training=True)
            loss   = self.loss_fn(y, logits)
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
        return {"loss": loss}

    def get_weights(self):
        return {f"weight_{i}": v for i, v in enumerate(self.model.trainable_variables)}

    def set_weights(self, *new_ws):
        for v, nw in zip(self.model.trainable_variables, new_ws):
            v.assign(nw)
        return {"status": tf.constant(1, tf.int32)}

# Re-instantiate
fed_mod = FederatedModule(keras_model)


In [ ]:
# Cell 5: Build concrete functions for all four methods & save

export_dir = tempfile.mkdtemp()

# 1) Inference
infer_fn = tf.function(
    fed_mod.serving_default,
    input_signature=[tf.TensorSpec([None, WINDOW_SIZE, NUM_FEATURES], tf.float32)]
).get_concrete_function()

# 2) Train
train_fn = tf.function(
    fed_mod.train,
    input_signature=[
        tf.TensorSpec([None, WINDOW_SIZE, NUM_FEATURES], tf.float32),
        tf.TensorSpec([None],                        tf.int32),
    ]
).get_concrete_function()

# 3) Get weights (no inputs)
get_w_fn = tf.function(fed_mod.get_weights).get_concrete_function()

# 4) Set weights (specs per variable)
specs = [
    tf.TensorSpec(var.shape, var.dtype, name=f"weight_{i}")
    for i, var in enumerate(fed_mod.model.trainable_variables)
]
set_w_fn = tf.function(fed_mod.set_weights).get_concrete_function(*specs)

# Save with all four signatures
tf.saved_model.save(
    fed_mod,
    export_dir,
    signatures={
        "serving_default": infer_fn,
        "train":           train_fn,
        "get_weights":     get_w_fn,
        "set_weights":     set_w_fn,
    }
)
print("✅ Federated SavedModel written to:", export_dir)


AssertionError: Tried to export a function which references an 'untracked' resource. TensorFlow objects (e.g. tf.Variable) captured by functions must be 'tracked' by assigning them to an attribute of a tracked object or assigned to an attribute of the main object directly. See the information below:
	Function name = b'__inference_signature_wrapper_381561'
	Captured Tensor = <ResourceHandle(name="seed_generator_495/seed_generator_state/7374", device="/job:localhost/replica:0/task:0/device:CPU:0", container="Anonymous", type="tensorflow::Var", dtype and shapes : "[ DType enum: 9, Shape: [2] ]")>
	Trackable referencing this tensor = <tf.Variable 'seed_generator_495/seed_generator_state:0' shape=(2,) dtype=int64>
	Internal Tensor = Tensor("381241:0", shape=(), dtype=resource)

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]
tflite_model = converter.convert()

with open(TF_OUTPUT, 'wb') as f:
    f.write(tflite_model)
print("✅ Federated TFLite written to:", TF_OUTPUT)

# Cleanup
shutil.rmtree(export_dir)
